In [0]:
%run ../../_utils

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import when, to_date, col, dayofweek
from pyspark.sql.types import StringType, BooleanType


# Camada Silver

Na camada silver, limpezas e ajustes em dados devem ser aplicados

Caso seja possível, enriquecer os dados e extrair dados também deve acontecer nessa camada (minha definição)

In [0]:
tb_name = "silver.olist_order_reviews"
dataset_location = "olist_order_reviews_dataset"
target_location = f"dbfs:/FileStore/delta/silver/brazilian_ecommerce/{dataset_location}"

## 1 - Data ingestion

In [0]:
df = spark.read.table("bronze.olist_order_reviews") # leituira da delta table

In [0]:
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: timestamp (nullable = true)
 |-- review_answer_timestamp: timestamp (nullable = true)



In [0]:
display(df.take(10))

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,null,null,2018-01-18T00:00:00Z,2018-01-18T21:46:59Z
80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,null,null,2018-03-10T00:00:00Z,2018-03-11T03:05:13Z
228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,null,null,2018-02-17T00:00:00Z,2018-02-18T14:36:24Z
e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,null,Recebi bem antes do prazo estipulado.,2017-04-21T00:00:00Z,2017-04-21T22:02:06Z
f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,null,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01T00:00:00Z,2018-03-02T10:26:53Z
15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,null,null,2018-04-13T00:00:00Z,2018-04-16T00:39:37Z
07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,null,null,2017-07-16T00:00:00Z,2017-07-18T19:30:34Z
7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5,null,null,2018-08-14T00:00:00Z,2018-08-14T21:36:06Z
a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5,null,null,2017-05-17T00:00:00Z,2017-05-18T12:05:37Z
8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,2018-05-22T00:00:00Z,2018-05-23T16:45:47Z



## 2 - Data Cleaning

Aqui nessa table os dados já estão limpos e tratados. Só iremos converter um campo que deveria ser date, para date e escrever na camada silver


In [0]:
df = df.withColumn("review_creation_date", col("review_creation_date").cast("date"))

In [0]:
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: date (nullable = true)
 |-- review_answer_timestamp: timestamp (nullable = true)




## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)

[LOG] Saving silver.olist_order_reviews delta on dbfs:/FileStore/delta/silver/brazilian_ecommerce/olist_order_reviews_dataset... OK!



## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(table_name=tb_name, target_location=target_location)

[LOG] Creating delta table silver.olist_order_reviews on dbfs:/FileStore/delta/silver/brazilian_ecommerce/olist_order_reviews_dataset... OK!


In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from silver.olist_order_reviews limit 10

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,null,null,2018-01-18,2018-01-18T21:46:59Z
80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,null,null,2018-03-10,2018-03-11T03:05:13Z
228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,null,null,2018-02-17,2018-02-18T14:36:24Z
e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,null,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21T22:02:06Z
f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,null,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01,2018-03-02T10:26:53Z
15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,null,null,2018-04-13,2018-04-16T00:39:37Z
07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,null,null,2017-07-16,2017-07-18T19:30:34Z
7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5,null,null,2018-08-14,2018-08-14T21:36:06Z
a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5,null,null,2017-05-17,2017-05-18T12:05:37Z
8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,2018-05-22,2018-05-23T16:45:47Z
